In [26]:
import notebookjs
import glob
from notebookjs import execute_js
import random
from sklearn.calibration import calibration_curve
import sklearn
import shap
import pandas as pd
import numpy as np
import sys
from sklearn.neural_network import MLPClassifier

sys.path.insert(0, '../core/')
from calibrate import Calibrate

## User Simulation

In [22]:
np.histogram(X['Age'].values, bins=20) 

(array([2410, 3160, 2461, 3429, 3465, 2583, 3198, 2965, 1828, 2139, 1558,
        1033,  996,  599,  269,  227,  120,   54,   20,   47]),
 array([-1.5822062 , -1.3146154 , -1.0470244 , -0.7794335 , -0.51184255,
        -0.24425161,  0.02333932,  0.29093024,  0.55852115,  0.8261121 ,
         1.093703  ,  1.3612939 ,  1.6288849 ,  1.8964758 ,  2.1640668 ,
         2.4316578 ,  2.6992486 ,  2.9668396 ,  3.2344306 ,  3.5020213 ,
         3.7696123 ], dtype=float32))

In [27]:
X,y = shap.datasets.adult(display=True)


le = sklearn.preprocessing.LabelEncoder()
X['Sex'] = le.fit_transform(X['Sex'])
X = pd.get_dummies(X, columns=["Workclass", "Marital Status", "Occupation", "Relationship", "Race", "Country"])

scaler = sklearn.preprocessing.StandardScaler()
train_col_sacle = X[['Age','Education-Num','Hours per week']]
train_scaler_col = scaler.fit_transform(train_col_sacle)
train_scaler_col = pd.DataFrame(train_scaler_col,columns=train_col_sacle.columns)
X['Age'] = train_scaler_col['Age']
X['Education-Num'] = train_scaler_col['Education-Num']
X['Hours per week'] = train_scaler_col['Hours per week']

X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

clf = MLPClassifier(random_state=42, max_iter=100).fit(X_train, y_train)
y_preds = clf.predict_proba(X_valid)

# OHE labels
ohe = sklearn.preprocessing.OneHotEncoder()

y_valid_labels = ohe.fit_transform(y_valid.reshape(-1,1)).toarray()

In [28]:
c = Calibrate(data=X, predictions=y_preds, labels=y_valid_labels).calibrate()